<a href="https://colab.research.google.com/github/AlexOchoki/-EPL-2020-2021-Data-Analysis-Project/blob/main/Zambia_MD_Improved_Aug_with_docs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Zambia Master Data
## What this notebook appears to do
- Focused on Zambia master data.
- Cleans and standardizes channel/tier fields and related mappings.
- Joins multiple sources to enrich the dataset.
- Ensures uniqueness by identifying/removing duplicates.
- Exports cleaned outputs for downstream reporting.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Import libraries

In [ ]:
# Install necessary libraries for handling Excel files

!pip install --quiet openpyxl xlrd

import pandas as pd
import numpy as np
import re
import os
from pathlib import Path
from openpyxl import load_workbook

### Load data

- Loads data via pandas read_excel(dirty_file...)
- Loads data via pandas read_excel(previous_month_file...)
- Loads data via pandas read_excel(ired_file...)
- Assigns variables: base_dir, clean_dir, constants_dir, dirty_file, previous_month_file, ired_file, dirty_df, previous_month_df, ired_df


In [ ]:
# Define base folders
base_dir = Path('/content/drive/MyDrive/SNS/Zambia/2025/Aug/dirty')
clean_dir = Path('/content/drive/MyDrive/SNS/Zambia/2025/Aug/clean')
constants_dir = Path('/content/drive/MyDrive/SNS/Ethiopia/eth_constants/mappings')

#/content/drive/MyDrive/SNS/Zambia/2025/Aug/dirty/zambia_aug-dirty.xlsx
# Define file paths
dirty_file = base_dir / 'zambia_aug-dirty.xlsx'
previous_month_file = base_dir / 'jul_clean.xlsx'
ired_file = base_dir / 'ired_jul.xlsx'

# Read the file
dirty_df = pd.read_excel(dirty_file)
previous_month_df = pd.read_excel(previous_month_file)
ired_df = pd.read_excel(ired_file)

print("Files loaded successfully.")

Files loaded successfully.


### Cell 4: Feature engineering / mapping
- Creates/updates columns via: map
- Casts dtypes and/or parses dates.
- Saves data via excel.
- Assigns variables: missing_count, missing_pct, missing_summary, output_path


In [ ]:
print("===== dirty_df Exploration =====\n")

# 1. Shape of the DataFrame
print(f"Shape (Rows, Columns): {dirty_df.shape}")

# 2. Missing values: count and percentage
missing_count = dirty_df.isnull().sum().astype(int)
missing_pct = (dirty_df.isnull().mean() * 100).round(2)

# 3. Build summary DataFrame
missing_summary = pd.DataFrame({
    'Missing Count': missing_count,
    'Missing %': missing_pct.map(lambda x: f"{int(x) if x.is_integer() else x}%")
})

# 4. Display summary
print("\nMissing Values Summary:")
display(missing_summary)

# 5. Save to clean folder
clean_dir.mkdir(parents=True, exist_ok=True)
output_path = clean_dir / 'missing_summary.xlsx'
missing_summary.to_excel(output_path, index=True)

print(f"\n✅ Saved missing value summary to: {output_path}")
print("\n" + "=" * 50)

===== dirty_df Exploration =====

Shape (Rows, Columns): (20878, 54)

Missing Values Summary:


,Missing Count,Missing %
Country,0,0%
Client,0,0%
Sub Client Code,0,0%
Sub Client,0,0%
Customer/Outlet,18303,87.67%
MSD ACC,0,0%
Customer Tier,0,0%
Barcode,0,0%
Customer name,0,0%
Address 1,16898,80.94%



✅ Saved missing value summary to: /content/drive/MyDrive/SNS/Zambia/2025/Aug/clean/missing_summary.xlsx



### Check the important columns for nulls

- Assigns variables: important_columns, total_rows, null_count, null_pct



In [ ]:
# List of important columns to check
important_columns = ["MSD ACC", "Customer Tier", "Customer name", "Region", "Sales Organization Code",
    "Sales office Code", "BD territory code", "users", "Trade Channel", "Sub-Channel", "PICOS Channel & Tier"]

print("===== Key Column Null Check Summary =====\n")

total_rows = len(dirty_df)

for col in important_columns:
    null_count = dirty_df[col].isnull().sum()
    null_pct = (null_count / total_rows) * 100

    if null_count > 0:
        print(f"⚠️  '{col}': {null_count} nulls ({null_pct:.2f}%) — recheck or ask the Country Champion to reshare the file.")
    else:
        print(f"✅  '{col}' has no missing values.")

print("\n" + "=" * 50)

===== Key Column Null Check Summary =====

✅  'MSD ACC' has no missing values.
✅  'Customer Tier' has no missing values.
✅  'Customer name' has no missing values.
✅  'Region' has no missing values.
✅  'Sales Organization Code' has no missing values.
✅  'Sales office Code' has no missing values.
✅  'BD territory code' has no missing values.
✅  'users' has no missing values.
✅  'Trade Channel' has no missing values.
✅  'Sub-Channel' has no missing values.
✅  'PICOS Channel & Tier' has no missing values.



### Trim the dataset

- Trim the dataset



In [ ]:
# Strip leading/trailing spaces and reduce multiple internal spaces to one
for col in dirty_df.columns:
    dirty_df[col] = dirty_df[col].apply(
        lambda x: re.sub(r'\s+', ' ', str(x).strip()) if pd.notnull(x) else x
    )

### 🧮 Step: Clean and Validate 'MSD ACC' Column

This step ensures that:
- All `MSD ACC` values are cleaned and converted to whole numbers (integers)
- Any non-numeric or invalid entries are flagged for correction
- Duplicate MSD ACC are identified and summarized
- Assigns variables: invalid_entries, num_invalid, dirty_df, duplicate_counts, duplicates, total_duplicates

This is critical for maintaining master data integrity, especially where `MSD ACC` acts as a unique key.

In [ ]:
# === 🧼 Clean & Validate 'MSD Code' Column ===

if 'MSD ACC' in dirty_df.columns:
    # 1. Convert to numeric, invalids to NaN
    dirty_df['MSD ACC'] = pd.to_numeric(dirty_df['MSD ACC'], errors='coerce')

    # 2. Identify invalid (non-numeric) or decimal entries
    invalid_entries = dirty_df[dirty_df['MSD ACC'].isna() | (dirty_df['MSD ACC'] % 1 != 0)]
    num_invalid = len(invalid_entries)

    if num_invalid > 0:
        print(f"⚠️ {num_invalid} entries in 'MSD ACC' are either non-numeric or have decimal places.")
    else:
        print("✅ All 'MSD ACC' values are valid whole numbers.")

    # 3. Drop invalid or decimal MSD ACC (optional)
    dirty_df = dirty_df[dirty_df['MSD ACC'].notna() & (dirty_df['MSD ACC'] % 1 == 0)]

    # 4. Convert to nullable integer type
    dirty_df['MSD ACC'] = dirty_df['MSD ACC'].astype('Int64')

    # 5. Check for duplicates
    duplicate_counts = dirty_df['MSD ACC'].value_counts()
    duplicates = duplicate_counts[duplicate_counts > 1]
    total_duplicates = duplicates.sum()

    print("\n===== MSD ACC Summary =====")
    print(f"🔍 Unique duplicated MSD ACC: {len(duplicates)}")
    print(f"📌 Total duplicate rows: {total_duplicates}")

    if not duplicates.empty:
        print("\nTop 5 Duplicate MSD ACC:")
        print(duplicates.head())

    print("\n===== Final Summary =====")
    if total_duplicates > 0:
        print(f"⚠️ {total_duplicates} duplicate rows found — please verify.")
    else:
        print("✅ No duplicate MSD ACC found.")
else:
    print("❌ 'MSD ACC' column not found in the DataFrame.")

✅ All 'MSD ACC' values are valid whole numbers.

===== MSD ACC Summary =====
🔍 Unique duplicated MSD ACC: 0
📌 Total duplicate rows: 0

===== Final Summary =====
✅ No duplicate MSD ACC found.


### 🧹 Step: Remove Invalid and Duplicate MSD ACC

This step performs the following cleanup operations:
1. Removes rows where `MSD ACC` is missing or invalid (non-numeric or blank)
2. Deduplicates the `MSD ACC` column, keeping only the first occurrence
3. Provides a summary of rows removed and final row count after cleanup

In [ ]:
# === 🧼 Remove Invalid and Duplicate MSD ACCs ===

# Step 1: Remove rows with missing or invalid MSD ACCs
initial_rows = len(dirty_df)
dirty_df = dirty_df[dirty_df['MSD ACC'].notna()]
after_invalid_removal = len(dirty_df)
invalid_removed = initial_rows - after_invalid_removal

print(f"✅ Removed {invalid_removed} rows with invalid or non-numeric MSD ACCs.")

# Step 2: Remove duplicate MSD ACCs, keep first occurrence
before_dedup = len(dirty_df)
dirty_df = dirty_df.drop_duplicates(subset=['MSD ACC'], keep='first')
after_dedup = len(dirty_df)
duplicates_removed = before_dedup - after_dedup

print(f"✅ Removed {duplicates_removed} duplicate MSD ACC entries.")

# Final Summary
print("\n===== MSD ACC Cleaning Summary =====")
print(f"🧾 Initial rows:               {initial_rows}")
print(f"🗑️  Removed invalid MSD ACCs: {invalid_removed}")
print(f"♻️  Removed duplicate MSDs:    {duplicates_removed}")
print(f"📊 Final cleaned rows:         {after_dedup}")

✅ Removed 0 rows with invalid or non-numeric MSD ACCs.
✅ Removed 0 duplicate MSD ACC entries.

===== MSD ACC Cleaning Summary =====
🧾 Initial rows:               20878
🗑️  Removed invalid MSD ACCs: 0
♻️  Removed duplicate MSDs:    0
📊 Final cleaned rows:         20878


### 🔍 Step: Find Sales Organizations in dirty_df That Don't Match the Clean Mapping (Previous Month Clean)

This step identifies Sales Organizations in the `dirty_df` that don't match any of the standardized names in the reference Previous Month's.
It helps highlight misspellings, incorrect spacing, or formatting inconsistencies.

- Drops rows with NAs.
- Checks/removes duplicates.
- Touches columns: Sales Organization, SalesOrganizationName, _merge
- Assigns variables: dirty_names, prev_names, dirty_rsm_pairs, clean_rsm_pairs, invalid_rsm_mappings


In [ ]:
# 1) Standardize
dirty_names = dirty_df['Sales Organization'].astype(str).str.strip().str.lower()
prev_names  = previous_month_df['SalesOrganizationName'].astype(str).str.strip().str.lower()

# 2) Convert to DataFrames (so merge works)
dirty_rsm_pairs = pd.DataFrame({'Sales Organization': dirty_names.dropna().drop_duplicates()})
clean_rsm_pairs = pd.DataFrame({'Sales Organization': prev_names.dropna().drop_duplicates()})

# 3) Left-anti join
invalid_rsm_mappings = (
    dirty_rsm_pairs
        .merge(clean_rsm_pairs, on='Sales Organization', how='left', indicator=True)
        .query('_merge == "left_only"')
        .drop(columns=['_merge'])
        .reset_index(drop=True)
)

# 4) Output
print("===== 🔍 Sales Organization Mapping Validation Results =====\n")
if invalid_rsm_mappings.empty:
    print("✅ All Sales Organization names in dirty_df exist in previous_month_df.")
else:
    print("⚠️ Found Sales Organization names not in the reference (previous month):")
    display(invalid_rsm_mappings)


===== 🔍 Sales Organization Mapping Validation Results =====

✅ All Sales Organization names in dirty_df exist in previous_month_df.


### 🔍 Step: Find Sales Office in dirty_df That Don't Match the Clean Mapping (Previous Month Clean)

This step identifies Sales Office in the `dirty_df` that don't match any of the standardized names in the reference Previous Month's.
It helps highlight misspellings, incorrect spacing, or formatting inconsistencies.


In [ ]:
# 1) Standardize
dirty_offices = dirty_df['Sales Office'].astype(str).str.strip().str.lower()
prev_offices  = previous_month_df['SalesOfficeName'].astype(str).str.strip().str.lower()

# 2) Convert to DataFrames (so merge works)
dirty_office_pairs = pd.DataFrame({'Sales Office': dirty_offices.dropna().drop_duplicates()})
clean_office_pairs = pd.DataFrame({'Sales Office': prev_offices.dropna().drop_duplicates()})

# 3) Left-anti join
invalid_office_mappings = (
    dirty_office_pairs
        .merge(clean_office_pairs, on='Sales Office', how='left', indicator=True)
        .query('_merge == "left_only"')
        .drop(columns=['_merge'])
        .reset_index(drop=True)
)

# 4) Output
print("===== 🔍 Sales Office Mapping Validation Results =====\n")
if invalid_office_mappings.empty:
    print("✅ All Sales Office names in dirty_df exist in previous_month_df.")
else:
    print("⚠️ Found Sales Office names not in the reference (previous month):")
    display(invalid_office_mappings)


===== 🔍 Sales Office Mapping Validation Results =====

✅ All Sales Office names in dirty_df exist in previous_month_df.


### 🔍 Step: Find BD Territory in dirty_df That Don't Match the Clean Mapping (Previous Month Clean)

This step identifies BD Territory in the `dirty_df` that don't match any of the standardized names in the reference Previous Month's.
It helps highlight misspellings, incorrect spacing, or formatting inconsistencies.



In [ ]:
# 1) Standardize
dirty_bd_territory = dirty_df['BD territory'].astype(str).str.strip().str.lower()
prev_bd_territory  = previous_month_df['SalesTerritory'].astype(str).str.strip().str.lower()

# 2) Convert to DataFrames
dirty_bd_pairs = pd.DataFrame({'BD territory': dirty_bd_territory.dropna().drop_duplicates()})
clean_bd_pairs = pd.DataFrame({'BD territory': prev_bd_territory.dropna().drop_duplicates()})

# 3) Left-anti join
invalid_bd_territory = (
    dirty_bd_pairs
        .merge(clean_bd_pairs, on='BD territory', how='left', indicator=True)
        .query('_merge == "left_only"')
        .drop(columns=['_merge'])
        .reset_index(drop=True)
)

# 4) Output
print("===== 🔍 BD Territory Mapping Validation Results =====\n")
if invalid_bd_territory.empty:
    print("✅ All BD territory values in dirty_df exist in previous_month_df.")
else:
    print("⚠️ Found BD territory values not in the reference (previous month):")
    display(invalid_bd_territory)


===== 🔍 BD Territory Mapping Validation Results =====

⚠️ Found BD territory values not in the reference (previous month):


,BD territory
0,mary n'cube tembo
1,betty musonda
2,tepa likezo
3,paxina sebesi
4,kings nyirongo
5,steve kasinsa
6,philip sinyangwe
7,vacant liteta
8,killian musonda
9,marvin kalota


### 🔍 Step: Find Region in dirty_df That Don't Match the Clean Mapping (Previous Month Clean)

This step identifies Region in the `dirty_df` that don't match any of the standardized names in the reference Previous Month's.
It helps highlight misspellings, incorrect spacing, or formatting inconsistencies.


In [ ]:
# 1) Standardize
dirty_region = dirty_df['Region'].astype(str).str.strip().str.lower()
prev_region  = previous_month_df['RegionName'].astype(str).str.strip().str.lower()

# 2) Convert to DataFrames
dirty_region_pairs = pd.DataFrame({'Region': dirty_region.dropna().drop_duplicates()})
clean_region_pairs = pd.DataFrame({'Region': prev_region.dropna().drop_duplicates()})

# 3) Left-anti join
invalid_region = (
    dirty_region_pairs
        .merge(clean_region_pairs, on='Region', how='left', indicator=True)
        .query('_merge == "left_only"')
        .drop(columns=['_merge'])
        .reset_index(drop=True)
)

# 4) Output
print("===== 🔍 Region Mapping Validation Results =====\n")
if invalid_region.empty:
    print("✅ All Region values in dirty_df exist in previous_month_df.")
else:
    print("⚠️ Found Region values not in the reference (previous month):")
    display(invalid_region)


===== 🔍 Region Mapping Validation Results =====

⚠️ Found Region values not in the reference (previous month):


,Region
0,gt-south et


### Hierarchy Check
Checks for 0ne-one-mapping for the lower level values to higher level values.
BD Territory --> Sales Office --> Sales Organization

- Aggregates using groupby / agg.
- Checks/removes duplicates.
- Sorts rows for deterministic output.
- Touches columns: BD territory, Sales Office
- Assigns variables: bd_territory_asm_mapping, non_compliant_bds, conflict_df



In [ ]:
# === 🧭 Check for BD territory Reporting to Multiple Sales Offices ===

# Step 1: Count how many unique Sales Offices each BD territory maps to
bd_territory_asm_mapping = dirty_df.groupby('BD territory')['Sales Office'].nunique()

# Step 2: Filter BD territory names with more than one Sales Office
non_compliant_bds = bd_territory_asm_mapping[bd_territory_asm_mapping > 1]

# Step 3: Display results
print("===== BD Territory to Sales Office Hierarchy Check =====\n")
if not non_compliant_bds.empty:
    print("❌ Non-compliant BD territory entries found — BD territory reporting to multiple Sales Offices:\n")

    conflict_df = (
        dirty_df[dirty_df['BD territory'].isin(non_compliant_bds.index)]
        [['BD territory', 'Sales Office']]  # select the 2 columns from the DataFrame
        .drop_duplicates()
        .sort_values(by='BD territory')
        .reset_index(drop=True)
    )

    display(conflict_df)

    # Optional: export to Excel
    # conflict_df.to_excel(clean_dir / 'bd_sales_office_conflicts.xlsx', index=False)

else:
    print("✅ Hierarchy is compliant — Each BD Territory reports to only one Sales Office.")


===== BD Territory to Sales Office Hierarchy Check =====

❌ Non-compliant BD territory entries found — BD territory reporting to multiple Sales Offices:



,BD territory,Sales Office
0,Edna Moyo,Rose Manda
1,Edna Moyo,Carol Kabamba
2,Edna Moyo,Isabel Chimamu
3,Fortress Mweemba,Carol Kabamba
4,Fortress Mweemba,Namatama Sitali
5,Grace Kyungupengu,Dennis Chungu
6,Grace Kyungupengu,Angela Mwewa
7,Isaac chisulo,Isabel Chimamu
8,Isaac chisulo,Rose Manda
9,Kennedy Mwase,Rose Manda


### Hierarchy Check
Checks for 0ne-one-mapping for the lower level values to higher level values.
BD Territory --> Sales Office --> Sales Organization


- Aggregates using groupby / agg.
- Checks/removes duplicates.
- Sorts rows for deterministic output.
- Touches columns: Sales Office, Sales Organization
- Assigns variables: office_org_counts, non_compliant_offices, conflict_df


In [ ]:
# === 🧭 Check for Sales Office Reporting to Multiple Sales Organizations ===

# Step 1: Count how many unique Sales Organizations each Sales Office maps to
office_org_counts = dirty_df.groupby('Sales Office')['Sales Organization'].nunique()

# Step 2: Filter Sales Offices with more than one Sales Organization
non_compliant_offices = office_org_counts[office_org_counts > 1]

# Step 3: Display results
print("===== Sales Office → Sales Organization Hierarchy Check =====\n")
if not non_compliant_offices.empty:
    print("❌ Non-compliant entries found — Sales Office reporting to multiple Sales Organizations:\n")

    conflict_df = (
        dirty_df[dirty_df['Sales Office'].isin(non_compliant_offices.index)]
        [['Sales Office', 'Sales Organization']]
        .drop_duplicates()
        .sort_values(by=['Sales Office', 'Sales Organization'])
        .reset_index(drop=True)
    )

    display(conflict_df)

    # Optional: export to Excel
    # conflict_df.to_excel(clean_dir / 'sales_office_org_conflicts.xlsx', index=False)

else:
    print("✅ Hierarchy is compliant — Each Sales Office reports to only one Sales Organization.")


===== Sales Office → Sales Organization Hierarchy Check =====

❌ Non-compliant entries found — Sales Office reporting to multiple Sales Organizations:



,Sales Office,Sales Organization
0,Morton Musonda,Kabwe Musonda
1,Morton Musonda,Kamima Chikumo
2,Raphael Mukanda,Farai Mhende
3,Raphael Mukanda,Kamima Chikumo
4,Stephen Goma,Kabwe Musonda
5,Stephen Goma,Kamima Chikumo
6,Webby Simbeye,Farai Mhende
7,Webby Simbeye,Kamima Chikumo


### Column Length Check
Check the values with length >100 in the specific columns


In [ ]:
def check_column_value_lengths(df, columns_to_check, max_length=100):
    """
    Checks whether values in specified columns exceed a given max character length.
    Prints any violations along with MSD ACC and summary.

    Parameters:
    - df: pandas DataFrame (e.g., dirty_df)
    - columns_to_check: list of column names to validate
    - max_length: maximum allowed character length (default = 100)
    """
    for column in columns_to_check:
        # Filter rows where length exceeds max
        violations = df[df[column].astype(str).str.len() > max_length][['MSD ACC', column]]

        print(f"\n===== Checking '{column}' for values > {max_length} characters =====")
        if not violations.empty:
            print(f"❌ Found {len(violations)} values exceeding limit in '{column}':")
            for _, row in violations.iterrows():
                print(f"MSD ACC: {row['MSD ACC']} — Value: {row[column]}")
        else:
            print(f"✅ All values in '{column}' are within the {max_length}-character limit.")


# 🟢 Columns to check
columns_to_validate = ['Customer name', 'Address 1', 'Address 4']
check_column_value_lengths(dirty_df, columns_to_validate, max_length=100)



===== Checking 'Customer name' for values > 100 characters =====
✅ All values in 'Customer name' are within the 100-character limit.

===== Checking 'Address 1' for values > 100 characters =====
✅ All values in 'Address 1' are within the 100-character limit.

===== Checking 'Address 4' for values > 100 characters =====
✅ All values in 'Address 4' are within the 100-character limit.


### Trade Channel Comparison
This compares that trade channel and related columns with the previous month (or validated mapping) for correctness or inconsistencies


In [ ]:
def clean_and_compare_data_v2(dirty_df, previous_month_df, column_mapping, entity_name):
    """
    Compares a single entity column from dirty and previous month datasets.

    Parameters:
        dirty_df (DataFrame): Current month's raw data.
        previous_month_df (DataFrame): Previous month's clean data.
        column_mapping (tuple): (dirty_column, previous_column)
        entity_name (str): Logical entity name (e.g., 'Customer Tier')

    Returns:
        dict: Contains entity name, non-matching values, and count.
    """
    dirty_col, prev_col = column_mapping

    # Handle missing columns
    if dirty_col not in dirty_df.columns or prev_col not in previous_month_df.columns:
        print(f"❌ Column missing: '{dirty_col}' or '{prev_col}'")
        return {
            'entity': entity_name,
            'non_matching_values': pd.DataFrame(),
            'non_matching_count': 0
        }

    # Extract and clean relevant columns
    dirty_values = (
        dirty_df[[dirty_col]]
        .dropna()
        .drop_duplicates()
        .copy()
    )
    previous_values = (
        previous_month_df[[prev_col]]
        .dropna()
        .drop_duplicates()
        .copy()
    )

    # Normalize: lowercase + strip spaces
    dirty_values[dirty_col] = dirty_values[dirty_col].str.lower().str.strip()
    previous_values[prev_col] = previous_values[prev_col].str.lower().str.strip()

    # Find unmatched values
    non_matches = dirty_values[~dirty_values[dirty_col].isin(previous_values[prev_col])]

    print(f"\n🔍 {entity_name} → New or inconsistent values in '{dirty_col}' not found in '{prev_col}':")
    if not non_matches.empty:
        print(non_matches)
    else:
        print("✅ All values match the previous month.")
    return {
        'entity': entity_name,
        'non_matching_values': non_matches,
        'non_matching_count': len(non_matches)
    }

### Trade channel comparison
- Check for inconstencies in trade channel column and outputs the results


In [ ]:
column_mappings = {
    'TradeChannelName': ('Trade Channel', 'Trade Channel'),
}

comparison_results = []

for entity, mapping in column_mappings.items():
    result = clean_and_compare_data_v2(dirty_df, ired_df, mapping, entity)
    comparison_results.append(result)

# Combine all mismatched records into one report
report_df = pd.concat(
    [r['non_matching_values'].assign(Entity=r['entity']) for r in comparison_results if r['non_matching_count'] > 0],
    ignore_index=True
)

print(f"\n📊 Total unmatched records across entities: {len(report_df)}")
# report_df.to_excel("unmatched_entities_report.xlsx", index=False)



🔍 TradeChannelName → New or inconsistent values in 'Trade Channel' not found in 'Trade Channel':
             Trade Channel
92                  horeca
919            supermarket
1001             pubs/bars
1032          pubs / clubs
5213    local/trade grocer
5223  general merchandiser
5229   eating and drinking
5233             wholesale
5249  full service vending
5274             superette
5276           hospitality
5428        general dealer
5712     convenience store
5779             petroleum
6082          recreational
7642             education
9208    hyper merchandiser

📊 Total unmatched records across entities: 17


### Standardize / Correct the wrong Trade Column values
- Checks for the assigns the correct values for identified wrong trade channel values and replaces them in the dataset


In [ ]:
# Normalize the column values
dirty_df['Trade Channel'] = dirty_df['Trade Channel'].astype(str).str.strip().str.lower()

# Define the mapping
picos_trade_channel_map = {
    'convinience and petrol': 'Convinience & Petrol',
    'convinience stores': 'Convinience & Petrol'
}

# Count occurrences before (on normalized values)
trade_channel_counts_before = dirty_df['Trade Channel'].value_counts(dropna=False)

# Apply the mapping
dirty_df['Trade Channel'] = dirty_df['Trade Channel'].replace(picos_trade_channel_map)

# Count individual replacements (how many of each bad key existed before)
trade_channel_replacements_count = {
    key: int(trade_channel_counts_before.get(key, 0)) for key in picos_trade_channel_map.keys()
}

# Display counts
print("\n'Trade Channel' replacements:")
for key, count in trade_channel_replacements_count.items():
    print(f"'{key}' replaced with '{picos_trade_channel_map[key]}': {count}")

# Total replacements
total_trade_channel_replacements = sum(trade_channel_replacements_count.values())
print(f"\nTotal replacements made: {total_trade_channel_replacements}")



'Trade Channel' replacements:
'convinience and petrol' replaced with 'Convinience & Petrol': 0
'convinience stores' replaced with 'Convinience & Petrol': 0

Total replacements made: 0


### PICOS Channel Check
Checks for inconsistencies in the PICOS Tier Channel against the previous month/mapping


In [ ]:
column_mappings = {'PICOS Channel & Tier': ('PICOS Channel & Tier', 'Sub-Channel')}

comparison_results = []

for entity, mapping in column_mappings.items():
    result = clean_and_compare_data_v2(dirty_df, ired_df, mapping, entity)
    comparison_results.append(result)

# Combine all mismatched records into one report
report_df = pd.concat(
    [r['non_matching_values'].assign(Entity=r['entity']) for r in comparison_results if r['non_matching_count'] > 0],
    ignore_index=True
)

print(f"\n📊 Total unmatched records across entities: {len(report_df)}")
# report_df.to_excel("unmatched_entities_report.xlsx", index=False)



🔍 PICOS Channel & Tier → New or inconsistent values in 'PICOS Channel & Tier' not found in 'Sub-Channel':
             PICOS Channel & Tier
0                   small grocery
1     local restaurants- matebeto
2                  medium grocery
9      pubs & bars- high end bars
45               local restaurant
131                     mini mart
251                           qsr
907                    local pubs
910                     local qsr
1001           local -pubs & bars
3054                 local liquor

📊 Total unmatched records across entities: 11


### Validation
Replaces the wrong values with the correct ones


In [ ]:
# Normalize the column values
dirty_df['PICOS Channel & Tier'] = dirty_df['PICOS Channel & Tier'].astype(str).str.strip().str.lower()

# Define the mapping
picos_channel_tier_map = {
    'small grocery': 'L&T Small Grocery',
    'local restaurants- matebeto': 'Local Restaurants - Matebeto',
    'medium grocery': 'L&T Medium Grocery',
    'pubs & bars- high end bars': 'Pubs & Bars - High End Bars',
    'mini mart': 'L&T Mini Mart',
    'qsr': 'On Prem QSR',
    'local pubs': '',
    'local qsr': 'On Prem QSR',
    'local -pubs & bars': '',
    'local liquor': ''
    # Add more corrections here as needed
}

# Count occurrences before (on normalized values)
picos_channel_tier_counts_before = dirty_df['PICOS Channel & Tier'].value_counts(dropna=False)

# Apply the mapping
dirty_df['PICOS Channel & Tier'] = dirty_df['PICOS Channel & Tier'].replace(picos_channel_tier_map)

# Count individual replacements
picos_channel_tier_replacements_count = {
    key: int(picos_channel_tier_counts_before.get(key, 0)) for key in picos_channel_tier_map.keys()
}

# Display counts
print("\n'PICOS Channel & Tier' replacements:")
for key, count in picos_channel_tier_replacements_count.items():
    print(f"'{key}' replaced with '{picos_channel_tier_map[key]}': {count}")

# Total replacements
total_picos_channel_tier_replacements = sum(picos_channel_tier_replacements_count.values())
print(f"\nTotal replacements made: {total_picos_channel_tier_replacements}")



'PICOS Channel & Tier' replacements:
'small grocery' replaced with 'L&T Small Grocery': 12919
'local restaurants- matebeto' replaced with 'Local Restaurants - Matebeto': 1728
'medium grocery' replaced with 'L&T Medium Grocery': 2897
'pubs & bars- high end bars' replaced with 'Pubs & Bars - High End Bars': 180
'mini mart' replaced with 'L&T Mini Mart': 116
'qsr' replaced with 'On Prem QSR': 326
'local pubs' replaced with '': 10
'local qsr' replaced with 'On Prem QSR': 1
'local -pubs & bars' replaced with '': 66
'local liquor' replaced with '': 501

Total replacements made: 18744


### Checls the Customer Tier


In [ ]:
column_mappings = {'CustomerTierName': ('Customer Tier', 'Customer Tier')}

comparison_results = []

for entity, mapping in column_mappings.items():
    result = clean_and_compare_data_v2(dirty_df, ired_df, mapping, entity)
    comparison_results.append(result)

# Combine all mismatched records into one report
report_df = pd.concat(
    [r['non_matching_values'].assign(Entity=r['entity']) for r in comparison_results if r['non_matching_count'] > 0],
    ignore_index=True
)

print(f"\n📊 Total unmatched records across entities: {len(report_df)}")
# report_df.to_excel("unmatched_entities_report.xlsx", index=False)



🔍 CustomerTierName → New or inconsistent values in 'Customer Tier' not found in 'Customer Tier':
  Customer Tier
0           tin

📊 Total unmatched records across entities: 1


### Validation - Customer Tier
Replaces the identified wrong values with the correct ones


In [ ]:
# Normalize the column values
dirty_df['Customer Tier'] = dirty_df['Customer Tier'].astype(str).str.strip().str.lower()

# Define the mapping
customer_tier_map = {
    'tin': 'Bronze'
}

# Count occurrences before (on normalized values)
customer_tier_counts_before = dirty_df['Customer Tier'].value_counts(dropna=False)

# Apply the mapping
dirty_df['Customer Tier'] = dirty_df['Customer Tier'].replace(customer_tier_map)

# Count individual replacements
customer_tier_replacements_count = {
    key: int(customer_tier_counts_before.get(key, 0)) for key in customer_tier_map.keys()
}

# Display counts
print("\n'Customer Tier' replacements:")
for key, count in customer_tier_replacements_count.items():
    print(f"'{key}' replaced with '{customer_tier_map[key]}': {count}")

# Total replacements
total_customer_tier_replacements = sum(customer_tier_replacements_count.values())
print(f"\nTotal replacements made: {total_customer_tier_replacements}")



'Customer Tier' replacements:
'tin' replaced with 'Bronze': 11106

Total replacements made: 11106


### Create New useful columns
Creates new columns
- Market Segment
- UserName


In [ ]:
# --- Create Market Segment related columns ---

# 1) Combine PICOS Channel & Tier with Customer Tier
dirty_df['Market Segment'] = dirty_df['PICOS Channel & Tier'].astype(str) + " " + dirty_df['Customer Tier'].astype(str)

# 2) Copy into MarketSegmentName and MarketSegmentCode
dirty_df['MarketSegmentName'] = dirty_df['Market Segment']
dirty_df['MarketSegmentCode'] = dirty_df['Market Segment']

# --- Create UserName from 'users' column ---

def create_username(full_name):
    """
    Returns first letter of first name + full last name if there are at least 2 words,
    otherwise returns the original value.
    """
    if pd.isna(full_name):
        return None
    parts = str(full_name).strip().split()
    if len(parts) >= 2:
        return parts[0][0] + parts[1]
    else:
        return full_name

dirty_df['UserName'] = dirty_df['users'].apply(create_username)

### Save the clean file


In [ ]:
# ✅ Assign cleaned DataFrame to a new variable
zambia_clean = dirty_df.copy()

# ✅ Define output path
output_path = base_dir / 'zambia_clean_aug.xlsx'
# ✅ Save the cleaned DataFrame to Excel
zambia_clean.to_excel(output_path, index=False)

# ✅ Confirmation
print(f"✅ Cleaned DataFrame saved as 'eth_clean_aug' and written to:\n{output_path}")

✅ Cleaned DataFrame saved as 'eth_clean_aug' and written to:
/content/drive/MyDrive/SNS/Zambia/2025/Aug/dirty/zambia_clean_aug.xlsx


### Read the clean file


In [ ]:
zam_clean_md = pd.read_excel(base_dir / 'zambia_clean_aug.xlsx')

### Create Maxerience Upload File Export



In [ ]:
# Create a new DataFrame based on the mappings
clean_df = pd.DataFrame({
    'Country': 'Zambia',
    'SubClientCode': '84',
    'SubClient': 'CCBA Zambia',
    'CustomerCode': zam_clean_md['MSD ACC'],
    'Barcode': 'NO TAGS',
    'CustomerName': zam_clean_md['Customer name'],
    'Street': zam_clean_md['Street'],
    'Address2': '',
    'Address3': '',
    'Address4': zam_clean_md['Address 4'],
    'City': zam_clean_md['City'],
    'State': '',
    'PostalCode': '',
    'RegionCode': zam_clean_md['Region'],
    'RegionName': zam_clean_md['Region'],
    'Latitude': zam_clean_md['Geo coordinates Lat.'],
    'Longitude': zam_clean_md['Geo coordinates Long.'],
    'TimeZone': '(UTC+02:00) Harare, Pretoria',
    'PrimaryPhone': '',
    'MobilePhone': '',
    'Email': '',
    'SalesOrganizationCode': zam_clean_md['Sales Organization'],
    'SalesOrganizationName': zam_clean_md['Sales Organization'],
    'SalesOfficeCode': zam_clean_md['Sales Office'],
    'SalesOfficeName': zam_clean_md['Sales Office'],
    'SalesGroupCode': '',
    'SalesGroupName': '',
    'SalesTerritoryCode': zam_clean_md['BD territory'],
    'SalesTerritory': zam_clean_md['BD territory'],
    'TeleSellingTerritoryCode': '',
    'TeleSellingTerritoryName': '',
    'CreditApproverCode': '',
    'CreditApproverName': '',
    'MerchandizerCode': '',
    'MerchandizerName': '',
    'P1_TerritoryCode': '',
    'P1_TerritoryName': '',
    'P2_TerritoryCode': '',
    'P2_TerritoryName': '',
    'P3_TerritoryCode': '',
    'P3_TerritoryName': '',
    'P4_TerritoryCode': '',
    'P4_TerritoryName': '',
    'P5_TerritoryCode': '',
    'P5_TerritoryName': '',
    'ReserveRouteCode': '',
    'ReserveRouteName': '',
    'TradeChannelCode': zam_clean_md['Trade Channel'],
    'TradeChannelName': zam_clean_md['Trade Channel'],
    'SubTradeChannelCode': zam_clean_md['MarketSegmentName'],
    'SubTradeChannel': zam_clean_md['MarketSegmentName'],
    'LocalTradeChannelCode': '',
    'LocalTradeChannelName': '',
    'LocalSubTradeChannelCode': '',
    'LocalSubTradeChannelName': '',
    'TradeGroupCode': '',
    'TradeGroupName': '',
    'ChainCode': '',
    'ChainName': '',
    'CustomerTierCode': zam_clean_md['Customer Tier'],
    'CustomerTierName': zam_clean_md['Customer Tier'],
    'CPLCode': '',
    'CPLName': '',
    'ClusterCode': '',
    'ClusterName': '',
    'LocalClusterCode': '',
    'LocalClusterName': '',
    'MarketSegmentCode': zam_clean_md['MarketSegmentName'],
    'MarketSegmentName': zam_clean_md['MarketSegmentName'],
    'SegmentCode': '',
    'SegmentName': '',
    'EnvironmentCode': '',
    'EnvironmentName': '',
    'Assortment1': '',
    'Assortment2': '',
    'Assortment3': '',
    'Assortment4': '',
    'Assortment5': '',
    'ExtraFields': '',
    'IsActive': ''
})


# Save the new file
output_file_path = base_dir / 'zambia_maxerience_aug_2025.xlsx'
clean_df.to_excel(output_file_path, index=False)

print(f"Cleaned file saved to: {output_file_path}")

Cleaned file saved to: /content/drive/MyDrive/SNS/Zambia/2025/Aug/dirty/zambia_maxerience_aug_2025.xlsx


### Create Reporting File File Export

In [ ]:
# Create a new DataFrame with the desired structure and hardcoded values
new_df = pd.DataFrame({
    'Country': 'Zambia',
    'Client': 'CCBA Zambia',
    'Sub Client Code': '84',
    'Sub Client': 'Zambia Bottlers',
    'outletcode': zam_clean_md['MSD ACC'],
    'Customer Tier': zam_clean_md['Customer Tier'],
    'Barcode': 'NO TAG',
    'Customer name': zam_clean_md['Customer name'],
    'Address 1': zam_clean_md['Address 1'],
    'Street': zam_clean_md['Street'],
    'City': zam_clean_md['City'],
    'Region': zam_clean_md['Region'],
    'Country code': 'ZMB',
    'Geo coordinates Lat.': zam_clean_md['Geo coordinates Lat.'],
    'Geo coordinates Long.': zam_clean_md['Geo coordinates Long.'],
    'Sales OrganizationCode': zam_clean_md['Sales Organization'],
    'Sales Organization': zam_clean_md['Sales Organization'],
    'Sales officeCode': zam_clean_md['Sales Office'],
    'Sales Office': zam_clean_md['Sales Office'],
    'Sales groupCode': zam_clean_md['Sales Office'],
    'Sales Group': zam_clean_md['Sales Office'],
    'BD territory code': zam_clean_md['BD territory'],
    'BD territory': zam_clean_md['BD territory'],
    'users': zam_clean_md['UserName'],
    'Trade Channel': zam_clean_md['Trade Channel'],
    'Sub-Channel': zam_clean_md['PICOS Channel & Tier'],
    'PICOS Channel & Tier': zam_clean_md['MarketSegmentName'],
    'Date': '',
    'Month': '',
    'Year': '',
    'OCCD No': '',
    'OCCD Name': '',
    'Cooler Tier': '',
})

# Save the new file
new_file_path = base_dir /'zambia_ired_aug_2025.xlsx'
new_df.to_excel(new_file_path, index=False)

print(f"New file saved to: {new_file_path}")

New file saved to: /content/drive/MyDrive/SNS/Zambia/2025/Aug/dirty/zambia_ired_aug_2025.xlsx
